In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr
import sklearn.model_selection as ms
import math


%matplotlib inline

In [3]:
data = pd.read_json('https://raw.githubusercontent.com/wncc/learners-space/master/Machine%20Learning/MId%20Term%20Assignment/data.json')
data.head()

,0,1,2
0,-3.005382,-1.701191,1
1,-2.908105,-1.699589,1
2,-2.958954,-1.660940,1
3,-2.922840,-1.638722,1
4,-2.924240,-1.508256,1


In [4]:
label_counts = data.iloc[:,2].value_counts()
print(label_counts)

1    4679
0    1321
Name: 2, dtype: int64


In [5]:
# labels = np.array(data.iloc[:, 2])

In [6]:
X = data.iloc[:,:2]
Y = data.iloc[:,2:]
x_train,x_test,y_train,y_test = ms.train_test_split(X,Y, test_size = 0.2)

In [20]:
nr.seed(3432)
np.array(x_train).reshape(4800,2)
np.array(y_train).reshape(4800,1)



array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [1]])

In [21]:
print(x_train)

             0         1
5222  2.233427 -1.022874
2067 -1.076734  0.597008
648  -2.313592 -0.047871
4281  1.188463  1.038697
2001 -0.909774 -1.561372
...        ...       ...
2446 -0.649362 -0.136147
4039  1.059299 -0.383652
2413 -0.680865 -1.237630
4662  1.533398  0.486746
193  -2.880430  1.471616

[4800 rows x 2 columns]


In [9]:
# Function  
def f(A, coef):
  fun = np.dot(A,coef.reshape(5,1))
  return fun

In [10]:
# Hypothesis
def h(A, coef):
  return np.array(1/(1+(np.exp(-f(A,coef)))) )

In [11]:
# Loss function
def l(y, A, coef):   # y shouldn't be the predicted values
  return (-np.dot(y, np.log(h(A, coef))) - np.dot((1-y),np.log(1-h(A, coef))) )/y.shape[0]

In [12]:
# Gradient 
def g(A, coef, y):
  return np.dot(np.array(h(A, coef).reshape(4800,1) - y).reshape([1,4800]), A)/y.shape[0]
  # here we will get a 1x5 matrix consisting of the gradient wrt each of the coeff

In [13]:
# Gradient Descent 
def gradesc(A, y, coef,L):       # here coef is column vector 
  for i in range (250):
    coef = coef.reshape(5,1) - L*g(A, coef, y).reshape(5,1)
  return coef  


In [14]:

A0 = np.array(x_train[0]**4).reshape(1, 4800)
A1 = np.array(x_train[0]**3 * x_train[1]).reshape(1, 4800)
A2 = np.array(x_train[0]**2 * x_train[1]**2).reshape(1, 4800)
A3 = np.array(x_train[0]**1 * x_train[1]**3).reshape(1, 4800)
A4 = np.array(x_train[1]**4 ).reshape(1, 4800)
A = np.vstack((A0, A1, A2, A3, A4))
A = A.T
print(A)
A.shape

[[ 2.48821204e+01 -1.13956081e+01  5.21900388e+00 -2.39021923e+00
   1.09468169e+00]
 [ 1.34410451e+00 -7.45255474e-01  4.13216174e-01 -2.29112851e-01
   1.27034472e-01]
 [ 2.86514654e+01  5.92832091e-01  1.22663844e-02  2.53805738e-04
   5.25153545e-06]
 ...
 [ 2.14903734e-01  3.90637340e-01  7.10073895e-01  1.29072386e+00
   2.34618974e+00]
 [ 5.52864921e+00  1.75495738e+00  5.57075572e-01  1.76832324e-01
   5.61318292e-02]
 [ 6.88381664e+01 -3.51695281e+01  1.79681675e+01 -9.17996521e+00
   4.69005875e+00]]


(4800, 5)

In [15]:
coef = np.array([0.001, 0.001, 0.001, 0.001, 0.001])
type(coef)
coef.shape

(5,)

In [16]:
gradesc(A,y_train, coef, 0.001)

array([[ 0.20027882],
       [-0.00100257],
       [ 0.04265616],
       [-0.00192677],
       [-0.03138927]])

In [18]:
def predict(x_test, coef = gradesc(A,y_train, coef, 0.001) ):
  A0 = np.array(x_test[0]**4).reshape(1, 1200)
  A1 = np.array(x_test[0]**3 * x_test[1]).reshape(1, 1200)
  A2 = np.array(x_test[0]**2 * x_test[1]**2).reshape(1, 1200)
  A3 = np.array(x_test[0]**1 * x_test[1]**3).reshape(1, 1200)
  A4 = np.array(x_test[1]**4 ).reshape(1, 1200)
  A = np.vstack((A0, A1, A2, A3, A4))
  A = A.T
  return h(A, coef)

In [24]:
predict(x_test)

array([[0.56139143],
       [0.74505696],
       [0.47840768],
       ...,
       [0.50001503],
       [0.47740443],
       [0.87921658]])